In [98]:
import pandas as pd
from datetime import *
import json
import yfinance

df = pd.read_csv('spx2020.csv')
df = df.drop(columns=['Unnamed: 0'])
df['# of earnings'] = -1
df['% of positive impact of earnings'] = -1.0
df['# of positive surprise earnings'] = -1
df['% of positive impact of earnings when positive surprise'] = -1.0
df['# of negative surprise earnings'] = -1
df['% of positive impact of earnings when negative surprise'] = -1.0
df

,ticker,company,# of earnings,% of positive impact of earnings,# of positive surprise earnings,% of positive impact of earnings when positive surprise,# of negative surprise earnings,% of positive impact of earnings when negative surprise
0,VIAC,VIACOMCBS INC,-1,-1.0,-1,-1.0,-1,-1.0
1,SCHW,SCHWAB CHARLES CORP NEW,-1,-1.0,-1,-1.0,-1,-1.0
2,CSCO,CISCO SYSTEMS INC,-1,-1.0,-1,-1.0,-1,-1.0
3,HD,HOME DEPOT INC,-1,-1.0,-1,-1.0,-1,-1.0
4,QCOM,QUALCOMM INC,-1,-1.0,-1,-1.0,-1,-1.0
...,...,...,...,...,...,...,...,...
499,FOX,FOX CORP,-1,-1.0,-1,-1.0,-1,-1.0
500,GOOG,GOOGLE INC,-1,-1.0,-1,-1.0,-1,-1.0
501,GOOGL,GOOGLE INC,-1,-1.0,-1,-1.0,-1,-1.0
502,UA,UNDER ARMOUR INC,-1,-1.0,-1,-1.0,-1,-1.0


In [99]:
for i in range(len(df)):
    stock_abbr = df.iloc[i, 0]
    print(stock_abbr)
    !node get_earnings2.js {stock_abbr}
    f = open(f'{stock_abbr}.json')
    earnings = json.load(f)
    f.close()
    !rm {stock_abbr}.json

    start, end = '1995-01-01', '2022-12-31'
    stock_ticker = yfinance.Ticker(stock_abbr)
    stock = stock_ticker.history(start=start, end=end)
    stock['Close_t+1'] = stock.Close.shift(-1)

    count, count_up = 0, 0
    count_pos, count_pos_up = 0, 0
    count_neg, count_neg_up = 0, 0
    for j in range(len(earnings)):
        if earnings[j]['eps'] and earnings[j]['date'] in stock.index:
            count += 1
            if stock.loc[earnings[j]['date'], 'Close_t+1'] > stock.loc[earnings[j]['date'], 'Close']:
                count_up += 1
            if earnings[j]['estimated_eps']:
                if earnings[j]['estimated_eps'] < earnings[j]['eps']:
                    count_pos += 1
                    if stock.loc[earnings[j]['date'], 'Close_t+1'] > stock.loc[earnings[j]['date'], 'Close']:
                        count_pos_up += 1
                elif earnings[j]['estimated_eps'] > earnings[j]['eps']:
                    count_neg += 1
                    if stock.loc[earnings[j]['date'], 'Close_t+1'] > stock.loc[earnings[j]['date'], 'Close']:
                        count_neg_up += 1
    pos_impact = count_up / count if count > 0 else -1
    pos_pos_impact = count_pos_up / count_pos if count_pos > 0 else -1
    neg_pos_impact = count_neg_up / count_neg if count_neg > 0 else -1
    df.at[i, '# of earnings'] = count
    df.at[i, '% of positive impact of earnings'] = pos_impact
    df.at[i, '# of positive surprise earnings'] = count_pos
    df.at[i, '% of positive impact of earnings when positive surprise'] = pos_pos_impact
    df.at[i, '# of negative surprise earnings'] = count_neg
    df.at[i, '% of positive impact of earnings when negative surprise'] = neg_pos_impact
    print(f'earnings count: {count}')
    print(f'positive impact: {pos_impact}')
    print(f'positive surprise count: {count_pos}')
    print(f'positive impact on positive surprise: {pos_pos_impact}')
    print(f'negative surprise count: {count_neg}')
    print(f'positive impact on negative surprise: {neg_pos_impact}\n')

VIAC
- VIAC: No data found, symbol may be delisted
earnings count: 0
positive impact: -1
positive surprise count: 0
positive impact on positive surprise: -1
negative surprise count: 0
positive impact on negative surprise: -1

SCHW
earnings count: 112
positive impact: 0.5089285714285714
positive surprise count: 45
positive impact on positive surprise: 0.5333333333333333
negative surprise count: 19
positive impact on negative surprise: 0.2631578947368421

CSCO
earnings count: 112
positive impact: 0.5178571428571429
positive surprise count: 92
positive impact on positive surprise: 0.532608695652174
negative surprise count: 1
positive impact on negative surprise: 0.0

HD
earnings count: 112
positive impact: 0.4642857142857143
positive surprise count: 80
positive impact on positive surprise: 0.4375
negative surprise count: 9
positive impact on negative surprise: 0.6666666666666666

QCOM
earnings count: 110
positive impact: 0.5
positive surprise count: 76
positive impact on positive surprise

In [101]:
df.to_csv('earnings_impact.csv', index=False)

In [116]:
fdf = df[df['# of earnings'] > 0]
print(f"% of scraping success: {len(fdf) / len(df)}")
print(f"average % of positive impact of earnings: {fdf['% of positive impact of earnings'].mean()}")
pfdf = fdf[fdf['# of positive surprise earnings'] != 0]
print(f"average % of positive impact of earnings when positive surprise: {pfdf['% of positive impact of earnings when positive surprise'].mean()}")
nfdf = fdf[fdf['# of negative surprise earnings'] != 0]
print(f"average % of positive impact of earnings when negative surprise: {nfdf['% of positive impact of earnings when negative surprise'].mean()}")

pnfdf = pfdf[pfdf['# of negative surprise earnings'] != 0]
pnfdf = pnfdf.reset_index()
count, count_pln = 0, 0
for i in range(len(pnfdf)):
    count += 1
    if pnfdf.at[i, '% of positive impact of earnings when positive surprise'] > pnfdf.at[i, '% of positive impact of earnings when negative surprise']:
        count_pln += 1
print(f"% of stocks which the average % of positive impact of earnings when positive surprise is bigger than when negative surprise: {count_pln / count}")

% of scraping success: 0.8273809523809523
average % of positive impact of earnings: 0.5155757350333835
average % of positive impact of earnings when positive surprise: 0.5504389184469368
average % of positive impact of earnings when negative surprise: 0.41690654395229293
% of stocks which the average % of positive impact of earnings when positive surprise is bigger than when negative surprise: 0.7682926829268293
